In [30]:
file_path = "C:/Users/ancha/OneDrive/Desktop/BootCamp/DS/PS_20174392719_1491204439457_log.csv/PS_20174392719_1491204439457_log.csv"

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [33]:
df = pd.read_csv(file_path,sep=',')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [34]:
print("Missing values:\n", df.isnull().sum())

Missing values:
 step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


In [35]:
df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
encoder = OneHotEncoder()
encoded_types = encoder.fit_transform(df[['type']]).toarray()
encoded_type_df = pd.DataFrame(encoded_types, columns=encoder.get_feature_names_out(['type']))
df = pd.concat([df, encoded_type_df], axis=1).drop(['type'], axis=1)

In [36]:
scaler = StandardScaler()
df[['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']] = scaler.fit_transform(
    df[['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']])

In [37]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

In [38]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [45]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50


127253/127253 [==============================] - 1112s 9ms/step - loss: 0.0052 - accuracy: 0.9991 - val_loss: 0.0040 - val_accuracy: 0.9993
Epoch 2/50
127253/127253 [==============================] - 409s 3ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 0.0038 - val_accuracy: 0.9993
Epoch 3/50
127253/127253 [==============================] - 391s 3ms/step - loss: 0.0046 - accuracy: 0.9993 - val_loss: 0.0041 - val_accuracy: 0.9993
Epoch 4/50
127253/127253 [==============================] - 383s 3ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.0039 - val_accuracy: 0.9993
Epoch 5/50
127253/127253 [==============================] - 386s 3ms/step - loss: 0.0044 - accuracy: 0.9993 - val_loss: 0.0047 - val_accuracy: 0.9993


In [46]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

39767/39767 [==============================] - 83s 2ms/step - loss: 0.0034 - accuracy: 0.9993
Test Accuracy: 99.93%


In [47]:
model.save('fraud_detection_model.h5')

d:\ancha\Documents\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
